# This is the explorative notebook. We perform basic statistics and do some basic data modeling
## Here are some of the questions we might try to answer:
1. what is the focus of policy documents and what tools do they operationalise? For example, here we looked at a series of laws that deal with adapting to climate change (that is - how do we deal with what's already coming, e.g. extreme weather events) and asked a few questions.
2. what events are the laws focusing on - droughts? floods? hurricanes? wildfires? It's good to know, because it allows a gap analysis of what are some of the unaddressed challenges - if you are prone to floods, you really want a law that sorts out flood barriers and flood insurance, or planning and resource for wildfire prevention and control if you're in a fire risk area.
3. what kind of measures does the document include? are there taxes/subsidies? standards? basically, how does the policy work?

The findings from this are summarised in this policy brief. It was a horrible manual job. The subset of laws analysed here can be found if you filter the dataset to include the word adaptation in the 'framework ' field.


In [1]:
import os
import pandas as pd

## Define the constants

In [5]:
DATA_DIR = '../data'

## Read the data

In [6]:
laws_path = os.path.join(DATA_DIR, 'Climate Laws targets-23_03_2021.csv')
events_path = os.path.join(DATA_DIR, 'events-23_03_2021.csv')
legislations_path = os.path.join(DATA_DIR, 'legislations-23_03_2021.csv')

In [12]:
# Add engine 'python' to avoid UnicodeDecodeError, as the cell values are surrounded by double quotes
laws_pd = pd.read_csv(laws_path, sep=',', engine='python')
events = pd.read_csv(events_path, sep=',', engine='python')
legislations_path = pd.read_csv(legislations_path, sep=',', engine='python')

## Get some basic statistics and show the heads

### Climate laws targets

In [18]:
print("Columns", laws_pd.columns)
print("Shape", laws_pd.shape)

laws_pd.head()

Columns Index(['Id', 'Target type', 'Description', 'Ghg target', 'Year',
       'Base year period', 'Single year', 'Source', 'Geography',
       'Geography iso', 'Sector', 'Connected law ids', 'Scopes',
       'Visibility status'],
      dtype='object')
Shape (2161, 14)


,Id,Target type,Description,Ghg target,Year,Base year period,Single year,Source,Geography,Geography iso,Sector,Connected law ids,Scopes,Visibility status
0,8977,not_applicable,<div>Seaports and airports functioning within ...,False,2030.0,NaN,False,plan,Dominica,DMA,Disaster Risk Management (Drm),9751,NaN,published
1,8976,not_applicable,<div>90% of the population able to identify th...,False,2030.0,NaN,False,plan,Dominica,DMA,Other,9751,NaN,published
2,8975,not_applicable,"<div>100% of national budgeting, policies in p...",False,2030.0,NaN,False,plan,Dominica,DMA,Public Sector,9751,NaN,published
3,8974,not_applicable,<div>50% increase healthy coral reef coverage ...,False,2030.0,NaN,False,plan,Dominica,DMA,Adaptation,9751,NaN,published
4,8973,not_applicable,<div>100% of telecommunications restored withi...,False,NaN,NaN,False,plan,Dominica,DMA,Disaster Risk Management (Drm),9751,NaN,published


In [19]:
laws_pd.isnull().sum()

Id                      0
Target type           767
Description             3
Ghg target              0
Year                  312
Base year period      997
Single year             0
Source                  0
Geography               0
Geography iso           0
Sector                  0
Connected law ids    1131
Scopes                 15
Visibility status       0
dtype: int64

### Climate events